## **install packages**

In [4]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install tensorflow==1.12.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pil is already the newest version (5.1.0-1).
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
    100% |████████████████████████████████| 337kB 24.2MB/s 
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


    100% |████████████████████████████████| 83.1MB 540kB/s 
    100% |████████████████████████████████| 3.1MB 12.5MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

## **Running CUDA GPU**
Runtime->Change runtime type ->gpu save
in Notebook settings under Runtime tab on the upper 

https://medium.com/@iphoenix179/running-cuda-c-c-in-jupyter-or-how-to-run-nvcc-in-google-colab-663d33f53772

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

In [2]:
!nvidia-smi

Sun Apr 28 18:12:58 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## **Mounting Google Drive locally**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## **Clone tensorflow/models** 

In [7]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 25469, done.
remote: Total 25469 (delta 0), reused 0 (delta 0), pack-reused 25469
Receiving objects: 100% (25469/25469), 507.96 MiB | 33.55 MiB/s, done.
Resolving deltas: 100% (15248/15248), done.
Checking out files: 100% (2883/2883), done.


In [0]:
!mkdir object-detection

## **copy files from google drive data_colab directory to object detection directory**

In [0]:
!rsync -r /content/drive/My\ Drive/data_colab/  /content/object-detection


## **move portobuf to model/research and unzip**

downlod  protoc-3.7.1-linux-x86_64.zip

https://github.com/protocolbuffers/protobuf/releases


In [0]:
%cd /content/object-detection

In [0]:
!ls
!mv protobuf.zip /content/models/research
%cd /content/models/research
!unzip protobuf.zip

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

env: PYTHONPATH=/content/models/research:/content/models/research/slim


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

## **test installation correct or not **


In [0]:
!python object_detection/builders/model_builder_test.py

In [0]:
%cd  
!ls
%cd /content/
!ls

## **move datas and images to /models/research/object_detection**

In [0]:
!ls
%cd object-detection
!unzip images.zip

In [0]:
%cd /content/object-detection 
!ls
!mv images /content/models/research/object_detection
!rsync -r data/ /content/models/research/object_detection/data
!mv training /content/models/research/object_detection/training
!mv generate_tfrecord.py /content/models/research/object_detection

## ** install the object_dection library**

In [0]:
%cd /content/models/research/
!python3 setup.py install

## **create csv files train_labels.csv and  test_labels.csv**

In [0]:
!python3 xml_to_csv.py

## **create tfrecord file from csv file**

In [0]:
%cd /content/models/research/object_detection
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_dir=images/train
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv --output_path=data/test.record --image_dir=images/train

## **for transfer learning**
use  checkpoint and configuration files



In [0]:
%cd /content/models/research/object_detection
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
!tar -xvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

## **Train the model**
upto loss <1.00 or 0.500

In [0]:

!python3 legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config

## **export inference graph**
from the directory training select the checkpoint
and edit the command below 

In [0]:
%cd /content/models/research/object_detection/
!python3 export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path training/ssd_mobilenet_v1_pets.config \
    --trained_checkpoint_prefix training/model.ckpt-1383 \
    --output_directory hand_sign_out_inference_graph

## **copy output back to google drive**
copy the frozen_inference_graph.pb and other model datas from 
hand_sign_out_inference_graph directory to google drive 
data_colab/hand_sign_out_inference_graph directory

In [0]:
!rsync -r  hand_sign_out_inference_graph  /content/drive/My\ Drive/data_colab/hand_sign_out_inference_graph